In [54]:
import yfinance as yf
import networkx as nx
from networkx.algorithms.approximation import clique
import numpy as np
import itertools 
from itertools import combinations
import time
import pickle
import json
import ast

# Получение данных

In [55]:
stocks = ['SBER.ME', 'YNDX.ME', 'LKOH.ME', 'ROSN.ME', 'GAZP.ME', 'RTKM.ME', 'VTBR.ME', 
          'MVID.ME', 'MTSS.ME', 'MFOYY', 'AFLT.ME', 'AVAZ.ME', 'MGNT.ME', 'TATN.ME', 'NLMK.ME',
         'GMKN.ME', 'FIVE.ME', 'HYDR.ME', 'RSTI.ME', 'LNTA.ME', 'NVTK.ME', 'SNGSP.ME', 'PLZL.ME',
         'CHMF.ME', 'AFKS.ME', 'AGRO.ME', 'TCSG.ME', 'MSNG.ME', 'ALRS.ME', 'DSKY.ME']
data = yf.download(stocks,'2019-01-01','2020-01-01')

[*********************100%***********************]  30 of 30 completed


In [56]:
len(set(stocks))

30

In [57]:
data = data['Adj Close']

In [58]:
n_days = 253

In [59]:
data

,AFKS.ME,AFLT.ME,AGRO.ME,ALRS.ME,AVAZ.ME,CHMF.ME,DSKY.ME,FIVE.ME,GAZP.ME,GMKN.ME,...,PLZL.ME,ROSN.ME,RSTI.ME,RTKM.ME,SBER.ME,SNGSP.ME,TATN.ME,TCSG.ME,VTBR.ME,YNDX.ME
Date,,,,,,,,,,,,,,,,,,,,,
2018-12-31,NaN,NaN,NaN,NaN,12.28,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-02,NaN,NaN,NaN,NaN,12.28,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-03,7.907535,99.372185,798.754089,86.439995,12.28,743.841919,75.032631,1819.751099,145.373688,10723.402344,...,4901.947754,381.296051,0.727189,62.661476,131.953506,31.177141,541.361633,NaN,0.032529,1859.5
2019-01-04,8.101215,98.980568,776.788269,87.788597,12.28,756.183533,75.065727,1864.670898,148.043396,10786.953125,...,4889.922852,385.369995,0.727838,62.824661,134.776184,30.898417,545.726196,NaN,0.032919,1916.0
2019-01-07,NaN,NaN,NaN,NaN,12.28,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-25,14.889894,103.339996,629.509277,80.462219,NaN,831.847046,90.769897,2103.279053,253.250000,18315.066406,...,6787.377441,433.505920,1.326149,75.172623,187.054276,36.696037,660.100647,1313.080322,0.044941,2687.0
2019-12-26,14.930590,103.940002,634.505371,80.471809,12.28,834.721008,90.935066,2092.290527,254.100006,18252.472656,...,6946.444824,432.073639,1.290677,75.087059,187.205124,37.172863,663.887085,1304.285034,0.044961,2713.0
2019-12-27,15.037789,103.779999,643.898071,81.124725,12.28,842.624512,91.760910,2128.751953,257.109985,18320.849609,...,6951.309570,433.028473,1.315300,75.096565,190.085876,37.231247,668.540833,1307.883057,0.044844,2736.0


In [60]:
data = data.dropna(thresh=27)

In [61]:
pct_matrix = data.pct_change()
pct_matrix = pct_matrix.fillna(0)
cor_matrix = pct_matrix.corr()

In [62]:
stocks_ = cor_matrix.index.values
cor_matrix_np = np.asmatrix(cor_matrix)

# Без вычета среднего

## Нахождение знаковых клик

In [65]:
def get_sign_match(stocks, pct_matrix, num_mult_sign=2):
    stock_combinations = list(combinations(stocks, num_mult_sign))
    mult_sign_dict = dict((' '.join(el), 0) for el in stock_combinations)
    for st_mult_sign in stock_combinations:
        tmp_matrix = np.sign(pct_matrix[list(st_mult_sign)]).values
        mult_sign_dict[' '.join(st_mult_sign)] = sum((tmp_matrix == tmp_matrix[:, [0]]).all(axis=1))
    return mult_sign_dict

In [66]:
all_mult_sign_dict = dict((el, {}) for el in range(2, 11))
start_time = time.time()
for i in range(2, 9):
    cur_time = time.time()
    mult_sign_dict = get_sign_match(stocks, pct_matrix, i)
    print("{}: {} seconds".format(i, (time.time() - cur_time)))
    all_mult_sign_dict[i] = mult_sign_dict
print("\nAll time: %s seconds" % (time.time() - start_time))

2: 0.7200725078582764 seconds
3: 6.534664154052734 seconds
4: 48.1564154624939 seconds
5: 241.6398253440857 seconds
6: 1012.8995773792267 seconds
7: 3657.3030610084534 seconds
8: 9876.759773731232 seconds

All time: 14844.024360895157 seconds


In [67]:
def convert(o):
    return int(o) 

In [68]:
json_dict = json.dumps(all_mult_sign_dict, default=convert)

f = open("rus_all_mult_sign_dict_.json","w")
f.write(json_dict)
f.close()

In [15]:
a_file = open("rus_all_mult_sign_dict_.json", "r")
output = a_file.read()
a_file.close()
loaded_dict = json.loads(output)

In [16]:
loaded_dict

{'2': {'SBER.ME YNDX.ME': 158,
  'SBER.ME LKOH.ME': 159,
  'SBER.ME ROSN.ME': 154,
  'SBER.ME GAZP.ME': 180,
  'SBER.ME RTKM.ME': 139,
  'SBER.ME VTBR.ME': 182,
  'SBER.ME MVID.ME': 149,
  'SBER.ME MTSS.ME': 170,
  'SBER.ME MFOYY': 14,
  'SBER.ME AFLT.ME': 140,
  'SBER.ME AVAZ.ME': 12,
  'SBER.ME MGNT.ME': 161,
  'SBER.ME TATN.ME': 169,
  'SBER.ME NLMK.ME': 150,
  'SBER.ME GMKN.ME': 164,
  'SBER.ME FIVE.ME': 155,
  'SBER.ME HYDR.ME': 169,
  'SBER.ME RSTI.ME': 155,
  'SBER.ME LNTA.ME': 146,
  'SBER.ME NVTK.ME': 159,
  'SBER.ME SNGSP.ME': 161,
  'SBER.ME PLZL.ME': 140,
  'SBER.ME CHMF.ME': 166,
  'SBER.ME AFKS.ME': 155,
  'SBER.ME AGRO.ME': 150,
  'SBER.ME TCSG.ME': 37,
  'SBER.ME MSNG.ME': 156,
  'SBER.ME ALRS.ME': 165,
  'SBER.ME DSKY.ME': 137,
  'YNDX.ME LKOH.ME': 143,
  'YNDX.ME ROSN.ME': 138,
  'YNDX.ME GAZP.ME': 155,
  'YNDX.ME RTKM.ME': 140,
  'YNDX.ME VTBR.ME': 157,
  'YNDX.ME MVID.ME': 126,
  'YNDX.ME MTSS.ME': 157,
  'YNDX.ME MFOYY': 14,
  'YNDX.ME AFLT.ME': 158,
  'YNDX.ME AVA

In [12]:
all_mult_sign_dict = loaded_dict

In [13]:
all_mult_sign_dict_top = {}

for i in all_mult_sign_dict.keys():
    sorted_dict = dict(sorted(all_mult_sign_dict[i].items(), key=lambda item: item[1], reverse=True)[:5])
    if len(sorted_dict):
        min_val = min(sorted_dict.values())
    
    for key, value in all_mult_sign_dict[i].items():
        if value == min_val and key not in sorted_dict:
            sorted_dict[key] = value
    print(i, ": ", sorted_dict)

    all_mult_sign_dict_top[i] = sorted_dict

2 :  {'MFOYY AVAZ.ME': 255, 'AVAZ.ME TCSG.ME': 218, 'MFOYY TCSG.ME': 212, 'NLMK.ME CHMF.ME': 195, 'SBER.ME VTBR.ME': 182}
3 :  {'MFOYY AVAZ.ME TCSG.ME': 212, 'SBER.ME GAZP.ME VTBR.ME': 138, 'TATN.ME NLMK.ME CHMF.ME': 136, 'SBER.ME GAZP.ME TATN.ME': 135, 'GAZP.ME TATN.ME GMKN.ME': 134}
4 :  {'SBER.ME GAZP.ME TATN.ME GMKN.ME': 109, 'SBER.ME GAZP.ME TATN.ME CHMF.ME': 108, 'SBER.ME GAZP.ME VTBR.ME TATN.ME': 106, 'SBER.ME GAZP.ME VTBR.ME CHMF.ME': 105, 'GAZP.ME TATN.ME NLMK.ME CHMF.ME': 105}
5 :  {'SBER.ME GAZP.ME VTBR.ME TATN.ME CHMF.ME': 88, 'SBER.ME GAZP.ME MTSS.ME TATN.ME GMKN.ME': 88, 'SBER.ME GAZP.ME TATN.ME NVTK.ME CHMF.ME': 87, 'SBER.ME GAZP.ME VTBR.ME TATN.ME GMKN.ME': 86, 'SBER.ME GAZP.ME MTSS.ME TATN.ME CHMF.ME': 86, 'SBER.ME GAZP.ME TATN.ME GMKN.ME CHMF.ME': 86}
6 :  {'SBER.ME GAZP.ME VTBR.ME MTSS.ME TATN.ME CHMF.ME': 73, 'SBER.ME GAZP.ME VTBR.ME TATN.ME GMKN.ME CHMF.ME': 72, 'SBER.ME GAZP.ME VTBR.ME TATN.ME NVTK.ME CHMF.ME': 72, 'SBER.ME GAZP.ME MTSS.ME TATN.ME GMKN.ME CHMF.ME'

## Клики по корреляции

In [14]:
cor_matrix_threshold = {}
thresholds = [0.3, 0.4, 0.5, 0.55, 0.6, 0.65, 0.7]

for tr in thresholds:
    tmp = cor_matrix[cor_matrix >= tr]
    tmp.values[[np.arange(tmp.shape[0])]*2] = 0
    tmp = tmp.fillna(0)
    cor_matrix_threshold[tr] = tmp

C:\Users\dimse\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  


## Топ "знаковых" клик

In [ ]:
all_mult_sign_dict_top = {}

for i in all_mult_sign_dict.keys():
    sorted_dict = dict(sorted(all_mult_sign_dict[i].items(), key=lambda item: item[1], reverse=True)[:5])
    if len(sorted_dict):
        min_val = min(sorted_dict.values())
    
    for key, value in all_mult_sign_dict[i].items():
        if value == min_val and key not in sorted_dict:
            sorted_dict[key] = value
    print(i, ": ", sorted_dict)

    all_mult_sign_dict_top[i] = sorted_dict

## Сравнение подходов

In [ ]:
for size, dict_top in all_mult_sign_dict_top.items():
    print("Size of sign cliques: {}\n".format(size))
    for sign_cliques, _ in dict_top.items():
        sign_cliques_list = sign_cliques.split()
        for threshold, cliques_threshold in cliques_threshold_dict.items():
            if len(sign_cliques_list) <= len(cliques_threshold):
                if(set(sign_cliques_list).issubset(set(cliques_threshold))): 
                    print("- Threshold {}".format(threshold))
                    print("sign_clique={}".format(sign_cliques_list))
                    print("cliques_threshold={}\n".format(cliques_threshold))
                    
    print("\n")

# С вычетом среднего

In [70]:
avg_data = data.copy()
avg_pct_matrix = avg_data.pct_change()
avg_pct_matrix = avg_pct_matrix.fillna(0)

In [75]:
avg_data.shape

(252, 30)

In [71]:
for col in avg_pct_matrix.columns:
    mean = avg_pct_matrix[col].mean()
    avg_pct_matrix[col] = avg_pct_matrix[col].apply(lambda x: x - mean)

In [72]:
avg_pct_matrix

,AFKS.ME,AFLT.ME,AGRO.ME,ALRS.ME,AVAZ.ME,CHMF.ME,DSKY.ME,FIVE.ME,GAZP.ME,GMKN.ME,...,PLZL.ME,ROSN.ME,RSTI.ME,RTKM.ME,SBER.ME,SNGSP.ME,TATN.ME,TCSG.ME,VTBR.ME,YNDX.ME
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-03,-0.002689,-0.000235,0.000799,0.000186,0.0,-0.000554,-0.000826,-0.000787,-0.002409,-0.002167,...,-0.001465,-0.000527,-0.002488,-0.000739,-0.001568,-0.000742,-0.000879,-0.000305,-0.001372,-0.001684
2019-01-04,0.021804,-0.004176,-0.026701,0.015788,0.0,0.016038,-0.000385,0.023897,0.015955,0.003760,...,-0.003918,0.010157,-0.001595,0.001865,0.019823,-0.009682,0.007183,-0.000305,0.010613,0.028700
2019-01-08,-0.009728,-0.000235,-0.013340,-0.028371,0.0,0.002605,-0.001047,-0.025413,-0.008001,0.002271,...,-0.030218,-0.016327,-0.013950,0.004045,-0.000259,-0.004350,0.009968,-0.000305,0.004983,0.003796
2019-01-09,0.011855,0.009260,0.020356,0.028063,0.0,0.020334,0.000276,-0.017253,0.010102,0.005527,...,0.025899,0.009152,0.018126,0.002798,0.028551,-0.007726,-0.008208,-0.000305,0.003220,0.015705
2019-01-10,0.000323,0.029943,-0.008792,0.015472,0.0,-0.003227,0.000936,-0.018924,-0.002659,0.006451,...,0.024886,0.009760,0.007358,0.000945,-0.002837,0.015148,0.019597,-0.000305,0.000056,0.011071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-24,-0.005217,-0.009843,-0.001747,-0.011905,0.0,-0.003349,0.012687,0.002766,-0.002919,-0.004504,...,0.009091,0.000360,0.021803,-0.000612,-0.002091,-0.007942,-0.010259,0.003834,-0.004812,-0.006055
2019-12-25,-0.002022,0.002287,0.005904,0.005706,0.0,-0.001848,-0.001635,-0.006924,-0.008765,0.011078,...,0.003722,0.005233,0.001584,0.002687,-0.004102,-0.006149,0.002960,0.002596,-0.009355,-0.002056
2019-12-26,0.000044,0.005571,0.008735,0.000306,0.0,0.002901,0.000993,-0.006012,0.000947,-0.005584,...,0.021971,-0.003831,-0.029236,-0.001878,-0.000762,0.012252,0.004857,-0.007003,-0.000937,0.007992


## Нахождение знаковых клик

In [73]:
avg_all_mult_sign_dict = dict((el, {}) for el in range(2, 11))
start_time = time.time()
for i in range(2, 9):
    cur_time = time.time()
    mult_sign_dict = get_sign_match(stocks, avg_pct_matrix, i)
    print("{}: {} seconds".format(i, (time.time() - cur_time)))
    avg_all_mult_sign_dict[i] = mult_sign_dict
print("\nAll time: %s seconds" % (time.time() - start_time))

2: 1.2327029705047607 seconds
3: 7.4799964427948 seconds
4: 45.56714963912964 seconds
5: 224.566481590271 seconds
6: 935.2384552955627 seconds
7: 3222.004695415497 seconds
8: 9237.965405464172 seconds

All time: 13674.057878017426 seconds


In [74]:
json_dict = json.dumps(avg_all_mult_sign_dict, default=convert)

f = open("rus_avg_all_mult_sign_dict.json","w")
f.write(json_dict)
f.close()

In [25]:
a_file = open("rus_avg_all_mult_sign_dict.json", "r")
output = a_file.read()
a_file.close()
loaded_dict = json.loads(output)
all_mult_sign_dict = loaded_dict

## Клики по корреляции

In [ ]:
avg_cor_matrix = avg_pct_matrix.corr()

In [ ]:
avg_cor_matrix_threshold = {}
thresholds = [0.1, 0.2, 0.3, 0.4, 0.5]

for tr in thresholds:
    tmp = avg_cor_matrix[avg_cor_matrix >= tr]
    tmp.values[[np.arange(tmp.shape[0])]*2] = 0
    tmp = tmp.fillna(0)
    avg_cor_matrix_threshold[tr] = tmp

In [ ]:
avg_cliques_threshold_dict = {}

for tr in thresholds:
    stocks_ = avg_cor_matrix.index.values
    avg_cor_matrix_np = np.asmatrix(avg_cor_matrix_threshold[tr])
    G = nx.from_numpy_matrix(avg_cor_matrix_np)

    G = nx.relabel_nodes(G,lambda x: stocks_[x])

    G.edges(data=True)
    print("Treshold", tr)
    cliques = clique.max_clique(G)
    print(cliques)
    avg_cliques_threshold_dict[tr] = cliques

## Топ "знаковых" клик

In [ ]:
avg_all_mult_sign_dict_top = {}

for i in avg_all_mult_sign_dict.keys():
    sorted_dict = dict(sorted(avg_all_mult_sign_dict[i].items(), key=lambda item: item[1], reverse=True)[:5])
    if len(sorted_dict):
        min_val = min(sorted_dict.values())
    
    for key, value in avg_all_mult_sign_dict[i].items():
        if value == min_val and key not in sorted_dict:
            sorted_dict[key] = value
    print(i, ": ", sorted_dict)

    avg_all_mult_sign_dict_top[i] = sorted_dict

## Сравнение подходов

In [ ]:
for size, dict_top in avg_all_mult_sign_dict_top.items():
    print("Size of sign cliques: {}\n".format(size))
    for sign_cliques, _ in dict_top.items():
        sign_cliques_list = sign_cliques.split()
        for threshold, cliques_threshold in avg_cliques_threshold_dict.items():
            if len(sign_cliques_list) <= len(cliques_threshold):
                if(set(sign_cliques_list).issubset(set(cliques_threshold))): 
                    print("- Threshold {}".format(threshold))
                    print("sign_clique={}".format(sign_cliques_list))
                    print("cliques_threshold={}\n".format(cliques_threshold))
                    
    print("\n")